In [9]:
import pandas as pd


In [10]:
df = pd.read_csv('../data/output/unlabeled_withBPAnnotations.csv')
df.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName
0,../DailyMedExtracter/prescription/temp_xml/47d...,115,123,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,http://purl.obolibrary.org/obo/DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
1,../DailyMedExtracter/prescription/temp_xml/47d...,453,461,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,http://purl.obolibrary.org/obo/DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
2,../DailyMedExtracter/prescription/temp_xml/47d...,667,675,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,http://purl.obolibrary.org/obo/DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
3,../DailyMedExtracter/prescription/temp_xml/47d...,4200,4208,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,http://purl.obolibrary.org/obo/DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
4,../DailyMedExtracter/prescription/temp_xml/47d...,4493,4501,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,http://purl.obolibrary.org/obo/DOID_552,DB00207,AZITHROMYCIN ANHYDROUS


In [11]:
print("The number of annotations from the bioportal annotator is:",  len(df))

The number of annotations from the bioportal annotator is: 8554


# Download and process UMLS to DOID mappings
# We use the propagated mappings here
url = 'https://raw.githubusercontent.com/dhimmel/disease-ontology/72614ade9f1cc5a5317b8f6836e1e464b31d5587/data/xrefs-prop-slim.tsv'
domap_df = pd.read_table(url)
domap_df.head()

In [12]:
#This is the updated DOID2UMLS mapping with 5974 unique DOID identifiers
domap_df = pd.read_csv("../data/input/DOID2UMLS.csv")
print(len(domap_df))
print(len(domap_df.drop_duplicates("doi_id")))
domap_df.head()

6634
5974


,doi_id,umls_id
0,DOID_0001816,C0018923
1,DOID_0001816,C0854893
2,DOID_175,C0282607
3,DOID_175,C0346845
4,DOID_175,C1290407


In [13]:
def trimName(url):
    return  url.replace("http://purl.obolibrary.org/obo/","")

df.DO_ID= df.DO_ID.apply(trimName)
df.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName
0,../DailyMedExtracter/prescription/temp_xml/47d...,115,123,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
1,../DailyMedExtracter/prescription/temp_xml/47d...,453,461,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
2,../DailyMedExtracter/prescription/temp_xml/47d...,667,675,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
3,../DailyMedExtracter/prescription/temp_xml/47d...,4200,4208,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS
4,../DailyMedExtracter/prescription/temp_xml/47d...,4493,4501,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS


In [14]:
df.drop_duplicates(['DO_ID', 'DB_ID'], inplace=True)

In [15]:
print(len(df))
print(len(df.drop_duplicates(['DO_ID', 'DB_ID'])))

4144
4144


In [16]:
domap_df = domap_df[['doi_id', 'umls_id']]   
domap_df.rename(columns={'doi_id':'DO_ID','umls_id':'disease'}, inplace=True)

domap_df.head()

,DO_ID,disease
0,DOID_0001816,C0018923
1,DOID_0001816,C0854893
2,DOID_175,C0282607
3,DOID_175,C0346845
4,DOID_175,C1290407


In [22]:
print(len(df))
print(len(domap_df))

4144
6634


In [25]:
#Merge the DOID mapping to the BPAnnotator
df2 = df.merge(domap_df, on=["DO_ID"], how='inner')
print(len(df2))
df2.head()

4736


,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,disease
0,../DailyMedExtracter/prescription/temp_xml/47d...,115,123,PREF,PNEUMONIA,Azithromycin for oral suspension USP is indica...,DOID_552,DB00207,AZITHROMYCIN ANHYDROUS,C0032285
1,../DailyMedExtracter/prescription/temp_xml/8eb...,3170,3178,PREF,PNEUMONIA,To reduce the development of drug-resistant ba...,DOID_552,DB00199,Erythromycin,C0032285
2,../DailyMedExtracter/prescription/temp_xml/685...,3435,3443,PREF,PNEUMONIA,[Symptomatic Trichomoniasis. Metronidazole tab...,DOID_552,DB00916,METRONIDAZOLE,C0032285
3,../DailyMedExtracter/prescription/temp_xml/8ee...,1017,1025,PREF,PNEUMONIA,"[ , \u200bTo reduce the d...",DOID_552,DB00417,PENICILLIN V,C0032285
4,../DailyMedExtracter/prescription/temp_xml/385...,445,453,PREF,PNEUMONIA,[Daptomycin for injection is indicated for the...,DOID_552,DB00080,Daptomycin,C0032285


In [26]:
df2.drop_duplicates(["DO_ID", "DB_ID"], inplace=True)

In [27]:
df2.rename(columns={'DB_ID':'Drug','disease':'Disease'},inplace=True)

In [28]:
df2.to_csv("../data/output/final_unlabeled.csv", index=False)

In [13]:
df = pd.read_csv("../data/contra/final_unlabeled.csv")

In [18]:
df.drop_duplicates(subset=["Disease", "Drug", "DO_ID"], inplace= True)

In [19]:
df

,Label_ID,From,To,Type,Annotation,Context,DO_ID,Drug,DrugName,Disease
0,../DailyMedExtracter/prescription/temp_xml/f39...,7,22,SYN,HYPERSENSITIVITY,Known hypersensitivity to Rose Bengal.,DOID_1205,DB14215,Rose Bengal AT,C0020517
1,../DailyMedExtracter/prescription/temp_xml/636...,1,16,SYN,HYPERSENSITIVITY,Hypersensitivity to Dapsone and/or its derivat...,DOID_1205,DB00250,DAPSONE,C0020517
2,../DailyMedExtracter/prescription/temp_xml/6c3...,1,16,SYN,HYPERSENSITIVITY,Hypersensitivity to any of the ingredients.,DOID_1205,DB00180,FLUNISOLIDE ANHYDROUS,C0020517
3,../DailyMedExtracter/prescription/temp_xml/34d...,41,56,SYN,HYPERSENSITIVITY,['Hypertension; toxemia; pregnancy; and hypers...,DOID_1205,DB00353,METHYLERGONOVINE,C0020517
4,../DailyMedExtracter/prescription/temp_xml/789...,1,16,SYN,HYPERSENSITIVITY,Hypersensitivity to any of the ingredients.,DOID_1205,DB00874,GUAIFENESIN,C0020517
5,../DailyMedExtracter/prescription/temp_xml/8a0...,1,16,SYN,HYPERSENSITIVITY,Hypersensitivity to any component of this prod...,DOID_1205,DB06794,LODOXAMIDE,C0020517
6,../DailyMedExtracter/prescription/temp_xml/8ab...,1,16,SYN,HYPERSENSITIVITY,Hypersensitivity to any component of this medi...,DOID_1205,DB00200,HYDROXOCOBALAMIN,C0020517
7,../DailyMedExtracter/prescription/temp_xml/8a2...,57,72,SYN,HYPERSENSITIVITY,"['\n ', 'CONTRAINDICATIONS...",DOID_1205,DB06824,TRIENTINE,C0020517
8,../DailyMedExtracter/prescription/temp_xml/eef...,15,30,SYN,HYPERSENSITIVITY,Anuria. Known hypersensitivity to chlorthalido...,DOID_1205,DB00310,CHLORTHALIDONE,C0020517
9,../DailyMedExtracter/prescription/temp_xml/b11...,31,46,SYN,HYPERSENSITIVITY,"[None, 'CONTRAINDICATIONS ', 'Hypersensitivity...",DOID_1205,DB00868,Benzonatate,C0020517


In [21]:
df.to_csv("../data/contra/final_unlabeled.csv", index=False)